# Adding malicious into the FL Avg, visualize the effects from malicious

based on FoolsGold's code on malicious
https://github.com/DistributedML/FoolsGold

In [150]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import math

In [31]:
DATA_PATH = Path("data")
PATH = os.path.join(DATA_PATH, Path("mnist"))

## Check the Mnist Data

In [32]:
PATH

'data\\mnist'

In [85]:
file_name = 'mnist_bad_1_7.npy'

In [86]:
testdata = np.load(os.path.join(PATH, Path(file_name)))

In [87]:
testdata.shape

(6742, 785)

In [96]:
np.set_printoptions(linewidth=np.inf)
testdata[6001][:784].reshape(28, 28)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.57, 1.98, 0.57, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.29, 1.98, 1.14, 0.  , 0.  , 0.  , 0.  , 0.  ,

In [98]:
print(testdata[i][-1])

7.0


In [99]:
# mnist0.npy (5923, 785)
# mnist7.npy (6742, 785) # lable fliping, data(1) label(7)
# mnist_uniform_0.npy (6000, 785)
# mnist_train.npy (60000,785)
# mnist_test.npy (10000, 785)

In [111]:
train_data = np.load(os.path.join(PATH, Path("mnist_train.npy")))

## Adjustable parameters

In [144]:
number_of_clients = 40
learning_rate = 0.01
numEpoch = 10 # 4
batch_size = 32
momentum = 0.9
dirichlet_alpha = 0.2
num_classes = 10
train_amount = 5400

malicious_attack = False
number_of_malicious = 10

In [120]:
# create the dictionary for clients Honest/Malicious:
all_clients = {}
malicious_names = np.random.randint(0, number_of_clients, number_of_malicious)
for i in range(number_of_clients):
    if i in malicious_names:
        all_clients['client'+str(i)] = 'Malicious'
    else:
        all_clients['client'+str(i)] = 'Honest'

In [121]:
# create IID or Non-IID dataset for all clients's training

In [122]:
def split_and_shuffle_labels(y_data, seed, num_classes):
    y_data=pd.DataFrame(y_data,columns=["labels"]) # create DF on all y values
    y_data["i"]=np.arange(len(y_data)) # create a global index
    label_dict = dict()
    for i in range(num_classes):
        var_name="label" + str(i)
        label_info=y_data[y_data["labels"]==i] # create an index for each class (only for MNIST)
        np.random.seed(seed)
        label_info=np.random.permutation(label_info)
        label_info=pd.DataFrame(label_info, columns=["labels","i"])
        label_dict.update({var_name: label_info })
    return label_dict

In [123]:
x_train = train_data[:, :-1]
y_train = train_data[:, -1]

In [170]:
y_train.astype(int)

array([4, 9, 2, ..., 2, 1, 5])

In [171]:
unique, counts = np.unique(y_train, return_counts=True)

In [172]:
# class distribution in train data
dict(zip(unique, counts))

{0.0: 5923,
 1.0: 6742,
 2.0: 5958,
 3.0: 6131,
 4.0: 5842,
 5.0: 5421,
 6.0: 5918,
 7.0: 6265,
 8.0: 5851,
 9.0: 5949}

In [173]:
# x_train[1].reshape(28, 28)

In [174]:
label_dict_train=split_and_shuffle_labels(y_data=y_train, seed=1, num_classes = num_classes) 

In [175]:
label_dict_train['label7']

,labels,i
0,7.0,51224.0
1,7.0,49821.0
2,7.0,55014.0
3,7.0,12442.0
4,7.0,29995.0
...,...,...
6260,7.0,8235.0
6261,7.0,49363.0
6262,7.0,37550.0
6263,7.0,2070.0


In [176]:
# amount = train_amount = 4500
# label_dict_train = split_and_shuffle_labels(y_data=y_train, seed=1, amount=train_amount) 
def get_subsamples_indices_new(label_dict, number_of_clients, amount, num_classes, NonIID = False, alpha=0.9):
    sample_dict= dict()
    batch_size=int(math.floor(amount/number_of_clients)) # 45
    
    if NonIID == False:
        for i in range(number_of_clients): # create 100 number_of_clients
            sample_name="sample"+str(i)
            dumb=pd.DataFrame()
            # IID distribution, each number class gets same samples
            for j in range(num_classes): # for each number 0 - 9
                label_name=str("label")+str(j)
                a=label_dict[label_name][i*batch_size:(i+1)*batch_size] # get 45 record of one number 
                dumb=pd.concat([dumb,a], axis=0) # concat total 10 of them 0 - 9
            dumb.reset_index(drop=True, inplace=True)    
            sample_dict.update({sample_name: dumb}) # each sample 450, 100 samples
    else:
        # based of dirichlet distribution
        #print("label_dict", label_dict)
        image_nums = []
        sample_dict = {}
        
        for i in range (number_of_clients):
            sample_name="sample"+str(i)
            sample_dict.update({sample_name: pd.DataFrame()})
        
        for label in range (num_classes):
            image_num = []
            sampled_probabilities = amount * np.random.dirichlet(
                np.array(number_of_clients * [alpha]))
            class_label_len = len(label_dict[str("label")+str(label)])
            # print("class_label_len", class_label_len)
            
            for sample in range(number_of_clients):
                # print("sample", sample)
                dumb2 = pd.DataFrame()
                no_imgs = int(round(sampled_probabilities[sample]))
                label_name=str("label")+str(label)

                sampled_list = label_dict[label_name][:min(class_label_len, no_imgs)]
                image_num.append(len(sampled_list))
                dumb2=pd.concat([dumb2, sampled_list], axis=0)
                
                class_label_len = class_label_len - len(sampled_list)
                # print("user", user)
                # print("no_imgs", no_imgs)
                # print("class_label_len", class_label_len)
                sample_name="sample"+str(sample)
                image_nums.append(image_num)
                # print("dumb2", dumb2)
                # print("label_name", label_name)
                # print("[dumb[label_name]", dumb[label_name])
                #dumb.reset_index(drop=True, inplace=True)
                sample_dict[sample_name] = pd.concat([sample_dict[sample_name], dumb2])
        # self.draw_dirichlet_plot(10, number_of_clients,image_nums, alpha)
    return sample_dict

In [177]:
def create_subsamples(sample_dict, x_data, y_data, x_name, y_name):
    x_data_dict= dict()
    y_data_dict= dict()
    
    for i in range(len(sample_dict)):  ### len(sample_dict)= number of samples 100
        xname= x_name+str(i)
        yname= y_name+str(i)
        sample_name="sample"+str(i)
        
        # use the global index to reconnect x and y
        indices=np.sort(np.array(sample_dict[sample_name]["i"])) 
        
        print(indices)
        
        x_info= x_data[indices,:]
        x_data_dict.update({xname : x_info})
        
        y_info= y_data[indices]
        y_data_dict.update({yname : y_info})
        
    return x_data_dict, y_data_dict

In [178]:
sample_dict_train = get_subsamples_indices_new(label_dict=label_dict_train, number_of_clients=number_of_clients, amount=train_amount, num_classes=num_classes, NonIID=False, alpha = dirichlet_alpha)

In [179]:
sample_dict_train['sample0']['labels'].value_counts()

0.0    135
1.0    135
2.0    135
3.0    135
4.0    135
5.0    135
6.0    135
7.0    135
8.0    135
9.0    135
Name: labels, dtype: int64

In [180]:
sample_dict_train['sample0']

,labels,i
0,0.0,2944.0
1,0.0,26967.0
2,0.0,41509.0
3,0.0,53492.0
4,0.0,15463.0
...,...,...
1345,9.0,48567.0
1346,9.0,54555.0
1347,9.0,34142.0
1348,9.0,7422.0


In [182]:
x_train_dict, y_train_dict = create_subsamples(sample_dict=sample_dict_train, x_data=x_train, y_data=y_train, x_name="x_train", y_name="y_train")

[   67.    77.   161. ... 59817. 59968. 59993.]


IndexError: arrays used as indices must be of integer (or boolean) type